In [2]:
# ==================== Colab Setup ====================
# 在 Google Colab 中运行此 notebook 的设置

# 检测是否在 Colab 环境
try:
    import google.colab
    IN_COLAB = True
    print("✓ 运行在 Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ 运行在本地环境")

if IN_COLAB:
    import os
    from pathlib import Path
    
    # 检查是否已经克隆
    if not Path("/content/echemistpy").exists():
        # 从 GitHub 克隆项目
        print("\n📥 克隆项目...")
        
        # 使用 Git LFS 跳过大文件（加速克隆）
        result = os.system("GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/liuchzzyy/echemistpy.git 2>&1")
        
        if result != 0:
            print("\n⚠️  Git 克隆失败！")
            print("\n可能的原因：")
            print("  1. 仓库还未推送到 GitHub")
            print("  2. 仓库是私有的（需要认证）")
            print("  3. 网络问题")
            print("\n请先确保代码已推送到 GitHub：")
            print("  git add .")
            print("  git commit -m '[FEATURE] 添加 Colab 支持'")
            print("  git push origin cl_version")
            raise RuntimeError("无法克隆仓库，请检查 GitHub 仓库状态")
    else:
        print("\n✓ 项目已存在，跳过克隆")
    
    # 切换到项目目录
    os.chdir("/content/echemistpy")
    print(f"✓ 当前目录: {os.getcwd()}")
    
    # 安装依赖
    print("\n📦 安装依赖...")
    os.system("pip install -q xarray numpy scipy pandas traitlets matplotlib scikit-learn h5py netCDF4 2>&1")
    
    # 安装项目本身
    if Path("pyproject.toml").exists() or Path("setup.py").exists():
        print("📦 安装 echemistpy...")
        result = os.system("pip install -q -e . 2>&1")
        if result == 0:
            print("✓ 安装完成！")
        else:
            print("⚠️  安装失败，但核心依赖已安装，可以继续运行")
    else:
        print("⚠️  未找到 pyproject.toml，跳过项目安装")
        # 手动添加 src 到路径
        import sys
        sys.path.insert(0, "/content/echemistpy/src")
        print("✓ 已添加 src 到 Python 路径")
    
    print("\n✓ 环境配置完成！")
    print("💡 提示: 下一步将从 OneDrive 自动下载测试数据（约 830 MB）")

✓ 运行在 Google Colab

📥 克隆项目...

⚠️  Git 克隆失败！

可能的原因：
  1. 仓库还未推送到 GitHub
  2. 仓库是私有的（需要认证）
  3. 网络问题

请先确保代码已推送到 GitHub：
  git add .
  git commit -m '[FEATURE] 添加 Colab 支持'
  git push origin cl_version


RuntimeError: 无法克隆仓库，请检查 GitHub 仓库状态

# 🚀 在 Google Colab 中运行此 Notebook

## 快速开始

1. **在 Colab 中打开此 notebook**：
   - 访问 [Google Colab](https://colab.research.google.com/)
   - `文件` → `打开 Notebook` → `GitHub`
   - 输入：`liuchzzyy/echemistpy`
   - 选择分支：`cl_version`
   - 选择：`notebooks/test_stxm_analyzer.ipynb`

2. **依次运行单元格**：
   - 第 2 个单元格：安装环境（约 1-2 分钟）
   - 第 5 个单元格：下载数据（约 3-5 分钟，830 MB）
   - 后续单元格：执行分析

3. **数据来源**：
   - 自动从 OneDrive/SharePoint 下载
   - 无需手动上传大文件

## 💡 注意事项

- **数据下载**: 测试数据约 830 MB，下载需要几分钟
- **运行时间**: 完整分析约需 5-10 分钟
- **GPU**: 不需要（但可以加速某些操作）

---

# Test STXMAnalyzer with Real Data

This notebook tests the `STXMAnalyzer` in `src/echemistpy/processing/analyzers/stxm.py` using a real TXM HDF5 file.

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# 检测是否在 Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 设置项目路径
if IN_COLAB:
    # Colab 中，假设已经 cd 到项目根目录
    project_root = Path.cwd()
else:
    # 本地环境，notebook 在 notebooks/ 子目录
    project_root = Path("..").resolve()
    if str(project_root / "src") not in sys.path:
        sys.path.append(str(project_root / "src"))

# Reload modules to pick up changes
import importlib
if 'echemistpy.processing.analyzers.stxm' in sys.modules:
    importlib.reload(sys.modules['echemistpy.processing.analyzers.stxm'])

from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.stxm import STXMAnalyzer

print(f"✓ 项目根目录: {project_root}")

## 下载数据文件

在 Colab 中从 OneDrive/SharePoint 下载测试数据。

In [ ]:
# ==================== 下载数据文件 ====================
import urllib.request
import shutil
from pathlib import Path

# OneDrive/SharePoint 数据链接（转换为直接下载链接）
ONEDRIVE_URL = "https://uab-my.sharepoint.com/:u:/g/personal/1615992_uab_cat/IQCiwUxTb7I-QpG_3-5KDu3VAZQiEam_jrJOLEVC0rDR6vk?e=rsxesw&download=1"

# 目标路径
data_dir = project_root / "docs" / "examples" / "TXM" / "αMnO2"
data_file = data_dir / "20230629_E1A_749.7x177.5y_specnorm_aliOF.hdf5.hdf5"

# 检查文件是否已存在
if data_file.exists():
    print(f"✓ 数据文件已存在: {data_file}")
    print(f"  大小: {data_file.stat().st_size / 1024 / 1024:.2f} MB")
else:
    print(f"📥 从 OneDrive 下载数据...")
    print(f"  URL: {ONEDRIVE_URL[:80]}...")
    
    # 创建目录
    data_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        # 下载文件
        print("  下载中... (这可能需要几分钟，文件约 830 MB)")
        
        # 使用临时文件避免部分下载
        temp_file = data_file.with_suffix('.tmp')
        
        def download_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            if total_size > 0:
                percent = min(downloaded * 100 / total_size, 100)
                mb_downloaded = downloaded / 1024 / 1024
                mb_total = total_size / 1024 / 1024
                print(f"\r  进度: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')
        
        urllib.request.urlretrieve(ONEDRIVE_URL, temp_file, download_progress)
        print()  # 换行
        
        # 移动到最终位置
        shutil.move(str(temp_file), str(data_file))
        
        print(f"✓ 下载完成: {data_file}")
        print(f"  大小: {data_file.stat().st_size / 1024 / 1024:.2f} MB")
        
    except Exception as e:
        print(f"✗ 下载失败: {e}")
        print("\n备用方案:")
        print("1. 手动下载文件:")
        print(f"   {ONEDRIVE_URL.replace('&download=1', '')}")
        print("2. 使用 Google Drive 上传数据")
        print("3. 上传到 Colab:")
        print("   from google.colab import files")
        print("   uploaded = files.upload()")
        
        # 清理临时文件
        if temp_file.exists():
            temp_file.unlink()
        
        raise

In [ ]:
# ==================== 加载数据 ====================
# 使用 alpha MnO2 示例数据
print(f"📂 加载数据: {data_file}")

if not data_file.exists():
    raise FileNotFoundError(f"数据文件不存在: {data_file}\n请先运行上面的下载单元格！")

raw_data, raw_info = load(data_file)

print("\n✓ 数据加载成功")
print(f"\n{'='*50}")
print("Raw Data Info:")
print(f"{'='*50}")
print(raw_info.to_dict())

print(f"\n{'='*50}")
print("Raw Data Structure:")
print(f"{'='*50}")
print(raw_data.data)

In [ ]:
# 2. Analyze
analyzer = STXMAnalyzer(
    pca_components=6,
    perform_clustering=True,
    n_clusters=4,
    # clustering_method="kmeans",
    perform_fitting=True,
)

# Use analyze() method which handles validation, preprocessing, computation, and metadata inheritance
result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 3. Inspect Results
print("\nAnalysis Info Parameters:")
print(result_info.parameters)

print("\nResult Dataset:")
ds_res = result_data.data
print(ds_res)


In [ ]:
# 4. Plot
ds_res = result_data.data

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# PCA Denoised Map (Sum)
if "denoised" in ds_res:
    if "energy_eV" in ds_res.denoised.dims:
        img_sum = ds_res.denoised.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Denoised Sum")
    else:
        ax[0].text(0.5, 0.5, "Denoised has no energy_eV dim")
elif "optical_density" in ds_res:
    if "energy_eV" in ds_res.optical_density.dims:
        img_sum = ds_res.optical_density.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Original Sum")
else:
    ax[0].text(0.5, 0.5, "No Image Data")

# Cluster Labels
if "cluster_labels" in ds_res:
    ax[1].imshow(ds_res.cluster_labels, cmap="tab10")
    ax[1].set_title("Cluster Labels")
else:
    ax[1].text(0.5, 0.5, "No Cluster Labels")

# Cluster Centroids / Spectra
if hasattr(result_info, "others") and "cluster_centroids" in result_info.others:
    centroids = result_info.others["cluster_centroids"]
    if "energy_eV" in ds_res.coords:
        energy = ds_res.energy_eV
        for i, cent in enumerate(centroids):
            ax[2].plot(energy, cent, label=f"Cluster {i}")

        ax[2].legend()
        ax[2].set_title("Cluster Spectra")
    else:
        ax[2].text(0.5, 0.5, "No energy_eV axis")
else:
    ax[2].text(0.5, 0.5, "No centroids")

plt.tight_layout()
plt.show()
